## Essential Imports

In [ ]:
import tensorflow as tf
from keras.regularizers import l2
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from keras.applications.resnet50 import ResNet50
from keras.applications.resnet50 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.models import load_model
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

## Essential Constants

In [ ]:
train_path = '/content/drive/My Drive/DM Course Project/Temp data(plot images)/temp data(plot images)/train/'
test_path = '/content/drive/My Drive/DM Course Project/Temp data(plot images)/temp data(plot images)/test/'

# useful for getting number of classes
folders = glob('/content/drive/My Drive/DM Course Project/Temp data(plot images)/temp data(plot images)/train/*')

num_classes=len(folders)

# re-size all the images to this
IMAGE_SIZE = [224, 224]

In [ ]:
num_classes

2

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


## Creating a new model

In [ ]:
model = Sequential()
model.add(ResNet50(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False))
model.add(Flatten())
model.add(Dense(units=128, activation='relu'))
model.add(Dense(1, kernel_regularizer=l2(0.01),activation='linear')) # Final Layer is SVM Layer
model.layers[0].trainable = False

In [ ]:
model.compile(optimizer='adam', loss='hinge', metrics=['accuracy',tf.keras.metrics.MeanAbsoluteError(),tf.keras.metrics.MeanSquaredError(),tf.keras.metrics.Hinge(),tf.keras.metrics.BinaryCrossentropy()])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Functional)        (None, 7, 7, 2048)        23587712  
_________________________________________________________________
flatten_1 (Flatten)          (None, 100352)            0         
_________________________________________________________________
dense_2 (Dense)              (None, 128)               12845184  
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 129       
Total params: 36,433,025
Trainable params: 12,845,313
Non-trainable params: 23,587,712
_________________________________________________________________


## Setting up training and testing data

In [ ]:
data_generator = ImageDataGenerator(preprocessing_function=preprocess_input)
train_set = data_generator.flow_from_directory(train_path,target_size = (224, 224),batch_size=10,class_mode = 'binary')
test_set = data_generator.flow_from_directory(test_path,target_size = (224, 224),batch_size=10,class_mode = 'binary')

Found 3000 images belonging to 2 classes.
Found 1400 images belonging to 2 classes.


In [ ]:
r = model.fit_generator(
  train_set,
  validation_data=test_set,
  epochs=20,
  steps_per_epoch=len(train_set),
  validation_steps=len(test_set)
)

f=open('/content/drive/My Drive/DM Course Project/Transfer Learning Model/TL-PI-Resnet50-stats.txt','w')
f.write(str(r.history))
f.close()

model.save('/content/drive/My Drive/DM Course Project/Transfer Learning Model/TL-PI-Resnet50-model.h5')

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/20
300/300 [==============================] - 1321s 4s/step - loss: 1.0763 - accuracy: 0.8130 - mean_absolute_error: 8.5219 - mean_squared_error: 151.0922 - hinge: 1.0612 - binary_crossentropy: 2.5801 - val_loss: 0.5013 - val_accuracy: 0.8814 - val_mean_absolute_error: 9.5402 - val_mean_squared_error: 165.5086 - val_hinge: 0.4892 - val_binary_crossentropy: 1.5644
Epoch 2/20
300/300 [==============================] - 27s 89ms/step - loss: 0.4374 - accuracy: 0.9000 - mean_absolute_error: 9.0309 - mean_squared_error: 139.1536 - hinge: 0.4267 - binary_crossentropy: 1.3520 - val_loss: 0.2401 - val_accuracy: 0.9300 - val_mean_absolute_error: 9.1355 - val_mean_squared_error: 135.0391 - val_hinge: 0.2305 - val_binary_crossentropy: 0.9016
Epoch 3/20
300/300 [==============================] - 27s 90ms/step - loss: 0.2017 - accuracy: 0.9407 - mean_absolute_error: 9.3241 - mean_squared_error: 142.0522 - hinge: 0.1928 - binary_crossentropy: 0.7539 - val_loss: 0.2705 - val_accuracy: 0.9450 -